In [9]:
# based on https://github.com/chkwon/TrafficAssignment.jl

include(joinpath("/home/jzh/Dropbox/Research/",
"Data-driven_estimation_inverse_optimization", 
"Experiments/InverseVIsTraffic/Julia_files/load_network_uni-class.jl"))

using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

ta_data = load_ta_network("East_Massachusetts_Jan_AM")

# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links);

In [10]:
function MSA(coeffs) 
    polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) 

    function BPR(x)
        bpr = similar(x)
        for i=1:length(bpr)
            bpr[i] = free_flow_time[i] * polyEval( coeffs, (x[i]/capacity[i]) ) 
        end
        return bpr
    end

    function all_or_nothing(travel_time)
        state = []
        path = []
        x = zeros(size(start_node))

        for r=1:size(travel_demand)[1]
            # for each origin node r, find shortest paths to all destination nodes
            state = dijkstra_shortest_paths(graph, travel_time, r)

            for s=1:size(travel_demand)[2]
                # for each destination node s, find the shortest-path vector
                # load travel demand
                x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
            end
        end

        return x
    end

    # Finding a starting feasible solution
    travel_time = BPR(zeros(number_of_links))
    xl = all_or_nothing(travel_time)

    max_iter_no = 1e3
    l = 1
    #average_excess_cost = 1
    tol = 1e-5

    while l < max_iter_no
        l += 1

        xl_old = xl

        # Finding yl
        travel_time = BPR(xl)

        yl = all_or_nothing(travel_time)

        xl = xl + (yl - xl)/l

        xl_new = xl

        relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

        if relative_gap < tol
            break
        end
    end
    
    return xl
end

MSA (generic function with 1 method)

In [11]:
# getting the coefficients of the costs
coeffs_dict_Jan_AM = readall("../temp_files/coeffs_dict_Jan_AM.json")
coeffs_dict_Jan_AM = JSON.parse(coeffs_dict_Jan_AM)

xl = Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}}()

deg_grid = [4:8]
c_grid = .5:.5:3.
lamb_grid = [10. .^(-3:4)]

for deg in deg_grid
    for c in c_grid
        for lam in lamb_grid
            coeffs_1 = coeffs_dict_Jan_AM["($(deg),$(c),$(lam),1)"]
            coeffs_2 = coeffs_dict_Jan_AM["($(deg),$(c),$(lam),2)"]
            coeffs_3 = coeffs_dict_Jan_AM["($(deg),$(c),$(lam),3)"]
            xl[(deg, c, lam, 1)] = MSA(coeffs_1)
            xl[(deg, c, lam, 2)] = MSA(coeffs_2)
            xl[(deg, c, lam, 3)] = MSA(coeffs_3)
        end
    end
end

In [12]:
lamb_grid

8-element Array{Float64,1}:
     0.001
     0.01 
     0.1  
     1.0  
    10.0  
   100.0  
  1000.0  
 10000.0  

In [13]:
outfile = open("../temp_files/uni-class_traffic_assignment_MSA_flows_Jan_AM.json", "w")

JSON.print(outfile, xl)

close(outfile)

In [14]:
xl

Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}} with 720 entries:
  (8,2.5,10.0,3)    => [1094.4933883959986,962.3951307099998,1374.7252542130018…
  (4,1.0,0.01,2)    => [1096.960809766999,962.3951307319995,1372.2578328420007,…
  (4,3.0,0.001,1)   => [1107.8721955790024,935.9313355879996,1359.5491754209975…
  (8,2.0,1000.0,3)  => [1092.0259670249998,963.8922851120002,1377.192675583999,…
  (6,1.5,0.1,1)     => [1092.0259670250007,963.8922852659991,1377.1926755840002…
  (6,1.5,100.0,1)   => [1092.0259670250011,963.892285035001,1377.1926755840007,…
  (4,2.0,1000.0,3)  => [1102.9373528370002,948.5491669090004,1364.4840181629997…
  (5,2.5,10.0,1)    => [1100.4699314660013,950.0463239399996,1366.9514395339986…
  (7,0.5,1.0,3)     => [1083.5820081499994,959.7819993760003,1388.2999996690012…
  (5,0.5,100.0,2)   => [1087.0911298490005,963.6327695480007,1383.3651569269987…
  (7,1.5,1.0,3)     => [1089.5585456539993,963.8922853649999,1379.6600969550022…
  (4,0.5,0.01,1)    => [1089.558551220

In [15]:
xl[(8,2.5,0.1,2)]

24-element Array{Float64,1}:
 1094.49 
  962.395
 1374.73 
 1393.14 
 1479.06 
 1532.03 
 1224.4  
 1353.03 
 1402.72 
 1782.87 
 1526.93 
 1120.72 
  917.977
 1091.32 
 1103.48 
  865.582
 1134.05 
 1030.17 
 1266.32 
 1308.48 
  733.761
 1412.9  
 1100.19 
 1204.53 